# Reliability calculations with many argumens in the limit state function

This example demonstrates how to perform reliability calculations with a limit state function that has multiple arguments, all following the same distribution function.

### Define model

First, let's import the necessary package:

In [26]:
from streams import ReliabilityProject, DistributionType, ReliabilityMethod, StandardNormal

We consider the following limit state function:

$Z = 1.9 - [\sum_{i=1}^{n}(a_i)+\sum_{j=1}^{m}(b_j)]$

This is a linear model involving two arrays, $a$ and $b$.

In [27]:
def linear(L: float, a: list[float], b:list[float]):
    s = L
    for i in range(len(a)):
        s -= a[i] + b[i]
    return s

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [28]:
project = ReliabilityProject()
project.model = linear

We assume that array $a$ consists of $25$ variables following a uniform distribution over the interval $[-1, 1]$. Array $b$ consists of $30$ variables, each normally distributed with a mean of $0$ and a standard deviation of $1$. This can be defined as follows:

In [29]:
project.variables["L"].distribution = DistributionType.deterministic
project.variables["L"].is_array = False
project.variables["L"].mean = 1.9

project.variables["a"].distribution = DistributionType.uniform
project.variables["a"].is_array = True # states that a is an array
project.variables["a"].array_size = 25
project.variables["a"].minimum = -1
project.variables["a"].maximum = 1

project.variables["b"].distribution = DistributionType.normal
project.variables["b"].is_array = True # states that b is an array
project.variables["b"].array_size = 30
project.variables["b"].mean = 0
project.variables["b"].deviation = 1

### Perform reliability calculations

We perform the reliability calculations using the form method.

In [30]:
project.settings.reliability_method = ReliabilityMethod.form
project.settings.relaxation_factor = 0.15
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02
project.run()

dp = project.design_point

def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")
        
    print(f"Model runs = {dp.total_model_runs}")

read_results(project.design_point)

Exception ignored on calling ctypes callback function: <function ReliabilityProject._performCallBack at 0x00000144A9D4A200>
Traceback (most recent call last):
  File "c:\Users\wojciech\AppData\Local\miniconda3\Lib\site-packages\streams\project.py", line 401, in _performCallBack
    ReliabilityProject._zmodel.run(values_list, output_values)
  File "c:\Users\wojciech\AppData\Local\miniconda3\Lib\site-packages\streams\project.py", line 137, in run
    z = ZModel._callback(*values);
        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\wojciech\AppData\Local\Temp\ipykernel_20684\4044445673.py", line 3, in linear
TypeError: object of type 'float' has no len()
Exception ignored on calling ctypes callback function: <function ReliabilityProject._performCallBack at 0x00000144A9D4A200>
Traceback (most recent call last):
  File "c:\Users\wojciech\AppData\Local\miniconda3\Lib\site-packages\streams\project.py", line 401, in _performCallBack
    ReliabilityProject._zmodel.run(values_list, output_values

Beta = 40.0
Probability of failure = 0.0
L: alpha = 0.0, x = 1.9
a: alpha = -0.7071067811865476, x = 1.0
b: alpha = -0.7071067811865476, x = 28.284271247461902
Not converged (convergence = nan > 0.02)
Model runs = 3
